In [ ]:
!pip3 install flotorch-eval crewai plotly opentelemetry-api opentelemetry-sdk pandas ragas duckduckgo-search openlit langchain_aws


In [1]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.sdk.trace.export.in_memory_span_exporter import InMemorySpanExporter


# Create an in-memory span exporter
memory_exporter = InMemorySpanExporter()
span_processor = SimpleSpanProcessor(memory_exporter)

# Set up the tracer provider and add the span processor
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(span_processor)
trace.set_tracer_provider(tracer_provider)

In [2]:
import openlit

# Initialize OpenLit - this will automatically instrument CrewAI when it's imported
openlit.init()

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


{
    "resource_metrics": [
        {
            "resource": {
                "attributes": {
                    "telemetry.sdk.language": "python",
                    "telemetry.sdk.name": "openlit",
                    "telemetry.sdk.version": "1.34.0",
                    "service.name": "default",
                    "deployment.environment": "default"
                },
                "schema_url": ""
            },
            "scope_metrics": [
                {
                    "scope": {
                        "name": "openlit.otel.metrics",
                        "version": "0.1.0",
                        "schema_url": "",
                        "attributes": null
                    },
                    "metrics": [
                        {
                            "name": "gen_ai.client.token.usage",
                            "description": "Measures number of input and output tokens used",
                            "unit": "{token}",
                 

In [3]:
from crewai.tools import tool
from duckduckgo_search import DDGS

@tool('DuckDuckGoSearch')
def search_tool(search_query: str):
    """Search the web for information on a given topic"""
    return DDGS().text(search_query, max_results=5)

@tool('SalesforceIntegration')
def salesforce_tool(soql_query: str):
    """Call Salesforce API to get data"""
    return "Salesforce Integration"

from crewai import LLM

model = LLM(
    # model="sagemaker/INSERT ENDPOINT NAME",
    model="bedrock/us.amazon.nova-pro-v1:0",
    temperature=0.7, max_tokens=4*1024,
)

from crewai import Agent, Task, Crew


writer = Agent(
        role="Writer",
        goal="You make GenAI concepts understandable for newbies exploring GenAI on AWS",
        backstory="You're an expert in writing crisp summaries about GenAI on AWS.",
        tools=[search_tool],
        llm=model
    )

task = Task(description=("What is {topic}?"),
            expected_output=("Compose a short summary that includes the answer."),
            agent=writer)

crew = Crew(
  agents=[writer],
  tasks=[task],
  share_crew=False
)

result = crew.kickoff({"topic": "AWS Bedrock"})
print(result)



{
    "name": "Crew Created",
    "context": {
        "trace_id": "0xff2f904880d407a9cd054b7c8b449d3a",
        "span_id": "0x551c67301da45cdd",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-06-09T00:18:27.025602Z",
    "end_time": "2025-06-09T00:18:27.026344Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "crewai_version": "0.126.0",
        "python_version": "3.13.0",
        "crew_key": "a7f38a97312c2a08d7161334f446c413",
        "crew_id": "dbb3f4b1-952a-41d9-8579-65b940961723",
        "crew_process": "sequential",
        "crew_memory": false,
        "crew_number_of_tasks": 1,
        "crew_number_of_agents": 1,
        "crew_fingerprint": "701b4b29-f5d1-498e-95c7-7afccf4877cb",
        "crew_fingerprint_created_at": "2025-06-08T17:18:27.024628",
        "crew_agents": "[{\"key\": \"18e63413ba6e2f4d81ec74e7660d93d9\", \"id\": \"6cf70835-7ba4-4304-8f19-8b478c4d0147\", \"role\":

In [4]:
spans = memory_exporter.get_finished_spans()

print("Number of spans:", len(spans))

Number of spans: 7


In [5]:
from langchain_aws import ChatBedrockConverse
from ragas.llms import LangchainLLMWrapper

bedrock_model = ChatBedrockConverse(
    region_name="us-east-1",
    endpoint_url=f"https://bedrock-runtime.us-east-1.amazonaws.com",
    model_id="us.amazon.nova-micro-v1:0"
)

llm = LangchainLLMWrapper(bedrock_model)



/Users/nanda/Work/workshops/agents-workshop/agent_eval/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from flotorch_eval.agent_eval.core.converter import TraceConverter

def create_trajectory(spans):
    converter = TraceConverter()
    trajectory = converter.from_spans(spans)
    return trajectory


In [8]:
trajectory = create_trajectory(spans)
trajectory

Trajectory(trace_id='ff2f904880d407a9cd054b7c8b449d3a', messages=[Message(role='user', content='What is AWS Bedrock?', tool_calls=[], timestamp=datetime.datetime(2025, 6, 8, 17, 18, 27, 31697)), Message(role='assistant', content='Amazon Bedrock is a fully managed service that makes it easy to use foundation models from third-party providers and Amazon. It allows users to build generative AI applications with a choice of foundation models from different AI companies, using a single API. Users can customize these models with their data, orchestrate multistep tasks, trace reasoning, and apply guardrails for responsible AI. Additionally, Amazon Bedrock enables the creation of generative AI workflows by connecting its features with other AWS services.', tool_calls=[], timestamp=datetime.datetime(2025, 6, 8, 17, 18, 29, 379326))], spans=[Span(span_id='551c67301da45cdd', trace_id='ff2f904880d407a9cd054b7c8b449d3a', parent_id=None, name='Crew Created', start_time=datetime.datetime(2025, 6, 8, 

In [ ]:
from flotorch_eval.agent_eval.core.evaluator import Evaluator
from flotorch_eval.agent_eval.metrics.langchain_metrics import (
    TrajectoryEvalWithLLMMetric,
    TrajectoryEvalWithoutLLMMetric,
)
from flotorch_eval.agent_eval.metrics.ragas_metrics import (
    AgentGoalAccuracyMetric,
    ToolCallAccuracyMetric,
)
from flotorch_eval.agent_eval.metrics.base import MetricConfig

evaluator = Evaluator([
    ToolCallAccuracyMetric(),
    AgentGoalAccuracyMetric(llm=llm,config=MetricConfig(
        metric_params={
            "reference_answer": "mazon Bedrock is a fully managed service that makes it easy to use foundation models from third-party providers and Amazon. It allows users to build generative AI applications with a choice of foundation models from different AI companies, using a single API. Users can customize these models with their data, orchestrate multistep tasks, trace reasoning, and apply guardrails for responsible AI. Additionally, Amazon Bedrock enables the creation of generative AI workflows by connecting its features with other AWS services."  # optional
        }
    )),
    TrajectoryEvalWithLLMMetric(llm = bedrock_model,config=MetricConfig(
        metric_params={
            "reference_outputs": [
    {"role": "user", "content": "What is AWS Bedrock?"},
    {
        "role": "assistant",
        "content": "To compose a poem about Amazon Bedrock, I first need to gather information about what Amazon Bedrock is. I will use the available tool to search for this information.",
        "tool_calls": [
            {
                "function": {
                    "name": "Search the web for information on a given topic",
                    "arguments": "{\"search_query\": \"Amazon Bedrock\"}"
                }
            }
        ]
    },
    {"role": "tool", "content": "{\"searchParameters\": {\"q\": \"Amazon Bedrock\", \"type\": \"search\", \"num\": 5, \"engine\": \"google\"}, \"organic\": [{\"title\": \"Amazon Bedrock - Generative AI - AWS\", \"link\": \"https://aws.amazon.com/bedrock/\", \"snippet\": \"Amazon Bedrock Data Automation streamlines the generation of valuable insights from unstructured multimodal content such as documents, images, audio, and videos ...\", \"position\": 1, \"sitelinks\": [{\"title\": \"Amazon Bedrock\", \"link\": \"https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html\"}, {\"title\": \"Amazon Bedrock Pricing\", \"link\": \"https://aws.amazon.com/bedrock/pricing/\"}, {\"title\": \"Amazon Bedrock Documentation\", \"link\": \"https://docs.aws.amazon.com/bedrock/\"}, {\"title\": \"Amazon Bedrock FAQs\", \"link\": \"https://aws.amazon.com/bedrock/faqs/\"}, {\"title\": \"Amazon Bedrock Agents\", \"link\": \"https://aws.amazon.com/bedrock/agents/\"}]}, {\"title\": \"Getting Started with Amazon Bedrock - AWS\", \"link\": \"https://aws.amazon.com/awstv/watch/6ff4cd6fa97/\", \"snippet\": \"So check the region that you're currently in, make sure it's a region that's supported by Bedrock. Then I'm gonna scroll to the bottom of this ...\", \"position\": 2}], \"relatedSearches\": [{\"query\": \"Amazon Bedrock pricing\"}, {\"query\": \"Amazon Bedrock documentation\"}, {\"query\": \"Amazon Bedrock Claude\"}, {\"query\": \"Amazon Bedrock logo\"}, {\"query\": \"Amazon Bedrock DeepSeek\"}], \"credits\": 1}"},
    {"role": "assistant", "content": "Based on the observation, I have learned that mazon Bedrock is a fully managed service that makes it easy to use foundation models from third-party providers and Amazon. It allows users to build generative AI applications with a choice of foundation models from different AI companies, using a single API. Users can customize these models with their data, orchestrate multistep tasks, trace reasoning, and apply guardrails for responsible AI. Additionally, Amazon Bedrock enables the creation of generative AI workflows by connecting its features with other AWS services."}
]
        }
    ))])

# Evaluate trajectory
results = await evaluator.evaluate(trajectory)



In [ ]:
results.scores